In [1]:
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
PROVINCES_JSON_FILENAME = 'provinces.json'
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoibGlzdGVuemNjIiwiYSI6ImNrMzU5MmpxZDAxMXEzbXQ0dnd4YTZ2NDAifQ.GohcgYXFsbDqfsi_7SXdpA'
HTML_FILENAME = 'a.html'
px.set_mapbox_access_token(MAPBOX_ACCESS_TOKEN)

LOG_FILENAME = "lastlog.log"
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

In [3]:
provinces_df = pd.read_json(PROVINCES_JSON_FILENAME)
provinces_df['log_confirmedCount'] = np.log10(provinces_df['confirmedCount']+1)
provinces_df

,cityName,confirmedCount,curedCount,deadCount,lat,lng,locationId,provinceName,suspectedCount,log_confirmedCount
0,武汉,10117,432,414,30.598467,114.311582,420100,湖北省,0,4.005095
1,孝感,1886,9,25,30.930689,113.922510,420900,湖北省,0,3.275772
2,黄冈,1807,62,29,30.459359,114.878490,421100,湖北省,0,3.257198
3,随州,834,9,9,31.696517,113.389450,421300,湖北省,0,2.921686
4,荆州,801,18,10,30.340842,112.245523,421000,湖北省,0,2.904174
...,...,...,...,...,...,...,...,...,...,...
405,第六师,1,0,0,43.799238,87.633473,0,新疆维吾尔自治区,0,0.301030
406,巴州,1,0,0,31.857686,106.775513,511902,新疆维吾尔自治区,0,0.301030
407,西宁,14,3,0,36.623385,101.784450,630100,青海省,0,1.176091
408,海北州,3,0,0,36.960663,100.907434,632200,青海省,0,0.602060


In [13]:
fig = px.scatter_mapbox(provinces_df, lat="lat", lon="lng",
                        color="log_confirmedCount",
                        # size="confirmedCount",
                        size_max=20,
                        color_continuous_scale=px.colors.carto.Redor,
                        zoom=2
                       )
with open(HTML_FILENAME, 'w') as f:
    f.write(fig.to_html())

In [34]:
fig = go.Figure(go.Scattermapbox(
        lat=provinces_df.lat,
        lon=provinces_df.lng,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9,
            color=provinces_df.log_confirmedCount,
            colorscale=px.colors.carto.Redor, #'Viridis', # one of plotly colorscales
            showscale=True
        ),
        text=provinces_df.confirmedCount,
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=MAPBOX_ACCESS_TOKEN,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=provinces_df.lat.mean(),
            lon=provinces_df.lng.mean()
        ),
        pitch=0,
        zoom=3,
    ),
)

with open(HTML_FILENAME, 'w') as f:
    f.write(fig.to_html())